In [1]:
# model_checkpoint = "bert-base-chinese"
# model_checkpoint = "schen/longformer-chinese-base-4096"
model_checkpoint = "voidful/albert_chinese_tiny"
batch_size = 2 # 每一批次的数量
num_labels = 2 # 多少分类，这里是二分类问题，积极和消极
output_dir = "/home/chenli/pre_model/roberta" # 模型保存路径
learning_rate = 1e-5 # 学习率
# weight_decay=0.01 # 学习率衰减，设置0.01即可。如果weight_decay设置太小，几乎就不起作用了。
num_train_epochs = 10 # 训练轮次，差不多设置为5左右。轮数不要设置太大。轮数设置的太大，Loss是下降了，但是微调的时候效果不是很好，有可能训练过头了

In [2]:
from datasets import load_dataset
from datasets import load_from_disk
# 加载一个评估标准，默认的评估标准
from datasets import load_metric

In [3]:
# 加载数据集
train_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/train_dataset.csv',split='train')
valid_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/valid_dataset.csv',split='train')
test_dataset = load_dataset('csv',data_files='../../data/MyDataset/travel_comment_data/test_dataset.csv',split='train')

Using custom data configuration default-be87a0d28564f2ae
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-be87a0d28564f2ae/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-e6970b6e6097a9c9
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-e6970b6e6097a9c9/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
Using custom data configuration default-8cb27c720d5c6049
Reusing dataset csv (/home/chenli/.cache/huggingface/datasets/csv/default-8cb27c720d5c6049/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [4]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 33858
})

In [5]:
valid_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4232
})

In [6]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4233
})

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/378 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
# 分词
def preprocess_function(data):
    return tokenizer(data['text'],padding='max_length', max_length=100,truncation=True)

In [11]:
train_encoded_dataset = train_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
val_encoded_dataset = valid_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])
test_encoded_dataset = test_dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])

  0%|          | 0/34 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [12]:
train_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 33858
})

In [13]:
val_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4232
})

In [14]:
test_encoded_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4233
})

In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

Some weights of the model checkpoint at voidful/albert_chinese_tiny were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at voidful/albert_chinese_tiny and are newly initialized: ['classifier.weight', 

In [17]:
metric_name = "accuracy"

args = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",# no step epoch
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = 1,
    num_train_epochs = num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # _n_gpu=2
)

In [18]:
import numpy as np
def compute_metrics(eval_preds):
    metric = load_metric('glue','mrpc')
    logits,labels = eval_preds # 预测值和真实值
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_encoded_dataset,
    eval_dataset=val_encoded_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### 20230522

In [20]:
trainer.train()

***** Running training *****
  Num examples = 33858
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84650
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.154300,0.182811,0.958176,0.971954
2,0.163500,0.177147,0.961484,0.973874
3,0.115700,0.180288,0.965501,0.976580
4,0.091100,0.192769,0.963374,0.975212
5,0.066700,0.210385,0.964792,0.976368
6,0.043700,0.249675,0.959594,0.972964
7,0.045200,0.250254,0.963138,0.975191
8,0.016500,0.295507,0.960775,0.973626
9,0.029600,0.302517,0.960302,0.973248
10,0.013100,0.300446,0.961248,0.973860


***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-8465
Configuration saved in /home/chenli/pre_model/roberta/checkpoint-8465/config.json
Model weights saved in /home/chenli/pre_model/roberta/checkpoint-8465/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/roberta/checkpoint-8465/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/roberta/checkpoint-8465/special_tokens_map.json
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-16930
Configuration saved in /home/chenli/pre_m

***** Running Evaluation *****
  Num examples = 4232
  Batch size = 2
Saving model checkpoint to /home/chenli/pre_model/roberta/checkpoint-84650
Configuration saved in /home/chenli/pre_model/roberta/checkpoint-84650/config.json
Model weights saved in /home/chenli/pre_model/roberta/checkpoint-84650/pytorch_model.bin
tokenizer config file saved in /home/chenli/pre_model/roberta/checkpoint-84650/tokenizer_config.json
Special tokens file saved in /home/chenli/pre_model/roberta/checkpoint-84650/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /home/chenli/pre_model/roberta/checkpoint-25395 (score: 0.9655009451795841).


TrainOutput(global_step=84650, training_loss=0.08152367166633516, metrics={'train_runtime': 3988.0969, 'train_samples_per_second': 84.898, 'train_steps_per_second': 21.226, 'total_flos': 266320933560000.0, 'train_loss': 0.08152367166633516, 'epoch': 10.0})

评估测试

In [21]:
trainer.evaluate(eval_dataset=test_encoded_dataset)

***** Running Evaluation *****
  Num examples = 4233
  Batch size = 2
/home/anaconda/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.20198166370391846,
 'eval_accuracy': 0.9605480746515473,
 'eval_f1': 0.9739347588574996,
 'eval_runtime': 40.1082,
 'eval_samples_per_second': 105.54,
 'eval_steps_per_second': 52.782,
 'epoch': 10.0}